Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/regression-part2-automated-ml.png)

# Tutorial: Use automated machine learning to predict taxi fares

In this tutorial, you use automated machine learning in Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this tutorial you learn the following tasks:

* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you donâ€™t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Complete the [setup tutorial](https://docs.microsoft.com/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) if you don't already have an Azure Machine Learning service workspace or notebook virtual machine.
* After you complete the setup tutorial, open the **tutorials/regression-automated-ml.ipynb** notebook using the same notebook server.

This tutorial is also available on [GitHub](https://github.com/Azure/MachineLearningNotebooks/tree/master/tutorials) if you wish to run it in your own [local environment](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/README.md#setup-using-a-local-conda-environment).

## Download and prepare data

Import the necessary packages. The Open Datasets package contains a class representing each data source (`NycTlcGreen` for example) to easily filter date parameters before downloading.

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from azureml.core import Dataset
from datetime import datetime
from dateutil.relativedelta import relativedelta

Begin by creating a dataframe to hold the taxi data. Then preview the data.

In [2]:
# Creating dataframe to hold the taxi data for one month
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))

green_taxi_df.head(10)

# green_taxi_dataset = Dataset.Tabular.from_parquet_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/green_taxi_data.parquet")
# green_taxi_df = green_taxi_dataset.to_pandas_dataframe()
# green_taxi_df.head(10)

[Info] read from /tmp/tmp3g19h4xb/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=1/part-00175-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2745-1.c000.snappy.parquet
[Info] read from /tmp/tmp84iq3xfh/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=2/part-00007-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2577-1.c000.snappy.parquet
[Info] read from /tmp/tmp0fzg6c03/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=3/part-00133-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2703-1.c000.snappy.parquet
[Info] read from /tmp/tmpbjcelwx0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=4/part-00073-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2643-1.c000.snappy.parquet
[Info] read from /tmp/tmpex8_chww/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2015/puMonth=5/part-00177-tid-4753095944193949832

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
409658,1,2015-01-19 17:17:51,2015-01-19 17:28:56,1,1.70,None,None,-73.979103,40.678123,-73.973709,...,1,9.5,0.0,0.5,0.3,2.00,0.0,NaN,12.30,1.0
1428782,2,2015-01-13 08:10:36,2015-01-13 08:26:22,1,2.79,None,None,-73.909424,40.664371,-73.952606,...,2,12.5,0.0,0.5,0.3,0.00,0.0,NaN,13.30,1.0
872918,2,2015-01-05 14:39:26,2015-01-05 14:50:45,1,1.89,None,None,-73.932365,40.690552,-73.958405,...,1,9.5,0.0,0.5,0.3,2.38,0.0,NaN,12.68,1.0
1360503,2,2015-01-07 09:26:59,2015-01-07 09:31:39,5,0.88,None,None,-73.986938,40.701611,-73.975174,...,2,5.5,0.0,0.5,0.3,0.00,0.0,NaN,6.30,1.0
175957,2,2015-01-14 09:37:53,2015-01-14 09:47:52,1,2.28,None,None,-73.942017,40.837849,-73.911873,...,1,10.0,0.0,0.5,0.3,2.00,0.0,NaN,12.80,1.0
1223115,2,2015-01-28 14:36:54,2015-01-28 14:50:02,1,1.87,None,None,-73.938751,40.845337,-73.954468,...,1,10.5,0.0,0.5,0.3,2.10,0.0,NaN,13.40,1.0
374167,2,2015-01-16 19:12:32,2015-01-16 19:19:51,1,1.70,None,None,-73.924820,40.761616,-73.920547,...,2,7.5,1.0,0.5,0.3,0.00,0.0,NaN,9.30,1.0
412741,2,2015-01-19 19:17:10,2015-01-19 19:26:32,1,1.23,None,None,-73.922188,40.754986,-73.911163,...,1,7.5,0.0,0.5,0.3,0.00,0.0,NaN,8.30,1.0
1033235,2,2015-01-29 13:03:02,2015-01-29 13:10:29,1,1.21,None,None,-73.948997,40.794392,-73.932358,...,2,7.0,0.0,0.5,0.3,0.00,0.0,NaN,7.80,1.0
1322519,2,2015-01-16 15:17:08,2015-01-16 15:21:31,5,0.89,None,None,-73.948326,40.795174,-73.938889,...,2,5.5,0.0,0.5,0.3,0.00,0.0,NaN,6.30,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day, and will allow the model to factor in time-based seasonality. 

Use the `apply()` function on the dataframe to iteratively apply the `build_time_features()` function to each row in the taxi data.

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
409658,1,2015-01-19 17:17:51,2015-01-19 17:28:56,1,1.70,None,None,-73.979103,40.678123,-73.973709,...,0.3,2.00,0.0,NaN,12.30,1.0,1,19,0,17
1428782,2,2015-01-13 08:10:36,2015-01-13 08:26:22,1,2.79,None,None,-73.909424,40.664371,-73.952606,...,0.3,0.00,0.0,NaN,13.30,1.0,1,13,1,8
872918,2,2015-01-05 14:39:26,2015-01-05 14:50:45,1,1.89,None,None,-73.932365,40.690552,-73.958405,...,0.3,2.38,0.0,NaN,12.68,1.0,1,5,0,14
1360503,2,2015-01-07 09:26:59,2015-01-07 09:31:39,5,0.88,None,None,-73.986938,40.701611,-73.975174,...,0.3,0.00,0.0,NaN,6.30,1.0,1,7,2,9
175957,2,2015-01-14 09:37:53,2015-01-14 09:47:52,1,2.28,None,None,-73.942017,40.837849,-73.911873,...,0.3,2.00,0.0,NaN,12.80,1.0,1,14,2,9
1223115,2,2015-01-28 14:36:54,2015-01-28 14:50:02,1,1.87,None,None,-73.938751,40.845337,-73.954468,...,0.3,2.10,0.0,NaN,13.40,1.0,1,28,2,14
374167,2,2015-01-16 19:12:32,2015-01-16 19:19:51,1,1.70,None,None,-73.924820,40.761616,-73.920547,...,0.3,0.00,0.0,NaN,9.30,1.0,1,16,4,19
412741,2,2015-01-19 19:17:10,2015-01-19 19:26:32,1,1.23,None,None,-73.922188,40.754986,-73.911163,...,0.3,0.00,0.0,NaN,8.30,1.0,1,19,0,19
1033235,2,2015-01-29 13:03:02,2015-01-29 13:10:29,1,1.21,None,None,-73.948997,40.794392,-73.932358,...,0.3,0.00,0.0,NaN,7.80,1.0,1,29,3,13
1322519,2,2015-01-16 15:17:08,2015-01-16 15:21:31,5,0.89,None,None,-73.948326,40.795174,-73.938889,...,0.3,0.00,0.0,NaN,6.30,1.0,1,16,4,15


Remove some of the columns that you won't need for training or additional feature building.

In [4]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
409658,1,1,1.70,-73.979103,40.678123,-73.973709,40.692467,12.30,1,19,0,17
1428782,2,1,2.79,-73.909424,40.664371,-73.952606,40.671776,13.30,1,13,1,8
872918,2,1,1.89,-73.932365,40.690552,-73.958405,40.683102,12.68,1,5,0,14
1360503,2,5,0.88,-73.986938,40.701611,-73.975174,40.698029,6.30,1,7,2,9
175957,2,1,2.28,-73.942017,40.837849,-73.911873,40.843166,12.80,1,14,2,9


### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [5]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.783583,1.376583,2.897795,-73.827228,40.689453,-73.847654,40.699580,14.889409,6.500000,15.165042,3.258417,13.581250
std,0.411810,1.054703,3.528351,2.821763,1.556094,2.524436,1.392235,16.264215,3.452124,8.427723,1.967540,6.796782
min,1.000000,0.000000,0.000000,-74.495300,0.000000,-74.507271,0.000000,-90.800000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.060000,-73.959818,40.697991,-73.967617,40.699326,7.800000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.900000,-73.944969,40.746693,-73.944454,40.747314,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.630000,-73.915926,40.803559,-73.908566,40.791357,17.760000,9.250000,22.000000,5.000000,19.000000
max,2.000000,6.000000,297.060000,0.000000,40.916683,0.000000,41.033119,1594.680000,12.000000,30.000000,6.000000,23.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [6]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [7]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23198.000000,23198.000000,23198.000000,23198.000000,23198.000000,23198.00000,23198.000000,23198.000000
mean,1.785197,1.380809,2.945379,14.817329,6.505173,15.16454,3.258126,13.587335
std,0.410694,1.061093,2.874671,10.739987,3.449666,8.42529,1.968085,6.810020
min,1.000000,1.000000,0.250000,0.250000,1.000000,1.00000,0.000000,0.000000
25%,2.000000,1.000000,1.100000,8.160000,4.000000,8.00000,2.000000,9.000000
50%,2.000000,1.000000,1.980000,11.300000,7.000000,15.00000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.800000,9.000000,22.00000,5.000000,19.000000
max,2.000000,6.000000,30.620000,350.300000,12.000000,30.00000,6.000000,23.000000


## Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [8]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

## Split the data into train and test sets

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(final_df, test_size=0.2, random_state=223)

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 20 minutes, but if you want a shorter run time, reduce the `experiment_timeout_hours` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|10|Time limit in minutes for each iteration. Increase this value for larger datasets that need more time for each iteration.|
|**experiment_timeout_hours**|0.3|Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**|True|Flag to enable early termination if the score is not improving in the short term.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**featurization**| auto | By using auto, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|

In [10]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 5,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": 'spearman_correlation',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [11]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             enable_onnx_compatible_models=True,
                             label_column_name="totalAmount",
                             **automl_settings)

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

In [12]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "Tutorial-NYCTaxi")
local_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Tutorial-NYCTaxi,AutoML_01a13230-c7d0-4166-b55c-7762ac02d26a,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high ca

2022-05-29:18:14:41,973 INFO     [explanation_client.py:334] Using default datastore for uploads


## Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [13]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [14]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Tutorial-NYCTaxi,
Id: AutoML_01a13230-c7d0-4166-b55c-7762ac02d26a_27,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=False, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=True, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             PreFittedSoftVotingRegressor(estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmregressor', LightGBMRegressor(min_data_in_leaf=20, n_jobs=-1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('23', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerW

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.

In [15]:
y_test = x_test.pop("totalAmount")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

[ 5.92902847  8.10490127  7.82266314 12.00620662 14.59967989  9.40585133
 26.58437771 24.83044719 19.42887594 22.30906913]


Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

3.5433134852702226

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.

In [17]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.13383289723854105

Model Accuracy:
0.8661671027614589


From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## Clean up resources

# Retrieve the best ONNX model

In [18]:
best_run, onnx_mdl = local_run.get_output(return_onnx_model=True)


## Save the ONNX Model

In [19]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Compute**.
1. Select the **Notebook VMs** tab in the compute page.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this automated machine learning tutorial, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using an automated regression model locally with custom parameters.
> * Explored and reviewed training results.

[Deploy your model](https://docs.microsoft.com/azure/machine-learning/service/tutorial-deploy-models-with-aml) with Azure Machine Learning service.